In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 100)

import time
import datetime

from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

### Подход 1: градиентный бустинг "в лоб"


**Задание 1.**
Считайте таблицу с признаками из файла features.csv. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
# Читаем таблицу с признаками.

data = pd.read_csv('data/features.csv', index_col='match_id')
data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,r2_level,r2_xp,r2_gold,r2_lh,r2_kills,r2_deaths,r2_items,r3_hero,r3_level,r3_xp,r3_gold,r3_lh,r3_kills,r3_deaths,r3_items,r4_hero,r4_level,r4_xp,r4_gold,r4_lh,r4_kills,r4_deaths,r4_items,r5_hero,r5_level,r5_xp,r5_gold,r5_lh,r5_kills,r5_deaths,r5_items,d1_hero,d1_level,d1_xp,d1_gold,d1_lh,d1_kills,d1_deaths,d1_items,d2_hero,d2_level,d2_xp,d2_gold,d2_lh,d2_kills,d2_deaths,d2_items,d3_hero,d3_level,d3_xp,d3_gold,d3_lh,d3_kills,d3_deaths,d3_items,d4_hero,d4_level,d4_xp,d4_gold,d4_lh,d4_kills,d4_deaths,d4_items,d5_hero,d5_level,d5_xp,d5_gold,d5_lh,d5_kills,d5_deaths,d5_items,first_blood_time,first_blood_team,first_blood_player1,first_blood_player2,radiant_bottle_time,radiant_courier_time,radiant_flying_courier_time,radiant_tpscroll_count,radiant_boots_count,radiant_ward_observer_count,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,67,3,842,991,10,0,0,4,29,5,1909,1143,10,0,0,8,20,3,757,741,6,0,0,7,105,3,732,658,4,0,1,11,4,3,1058,996,12,0,0,6,42,4,1085,986,12,0,0,4,21,5,2052,1536,23,0,0,6,37,3,742,500,2,0,0,8,84,3,958,1003,3,1,0,9,7.0,1.0,9.0,NaN,134.0,-80.0,244.0,2,2,2,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,49,4,1596,993,10,0,1,7,67,4,1506,1502,18,1,0,7,37,3,669,631,7,0,0,7,26,2,415,539,1,0,0,5,39,5,1960,1384,16,0,0,8,88,3,640,566,1,0,1,5,79,3,720,1350,2,2,0,12,7,2,440,583,0,0,0,7,12,4,1470,1622,24,0,0,9,54.0,1.0,7.0,NaN,173.0,-80.0,NaN,2,0,2,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,98,3,1314,775,6,0,0,6,20,3,1297,909,0,1,0,6,27,5,2360,2096,26,1,1,6,4,3,1395,1627,27,0,0,9,22,5,2305,2028,19,1,1,10,66,3,1024,959,19,0,1,10,86,3,755,620,3,0,0,8,29,4,1319,667,4,0,0,7,80,3,1350,1512,25,0,0,7,224.0,0.0,3.0,NaN,63.0,-82.0,NaN,2,5,2,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,30,2,539,539,1,0,0,6,75,5,2037,1139,15,0,0,6,37,2,591,499,0,0,0,6,41,3,712,1075,12,0,0,6,96,5,1878,1174,17,0,0,6,48,3,732,1468,22,0,0,10,15,4,1681,1051,11,0,0,7,102,2,674,537,1,0,0,7,20,2,510,499,0,0,0,7,NaN,NaN,NaN,NaN,208.0,-75.0,NaN,0,3,2,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,27,2,629,552,0,0,1,7,30,3,884,927,0,1,0,8,72,3,925,1439,16,1,0,11,93,4,1482,880,7,0,0,8,26,3,704,586,1,0,2,9,69,3,1169,1665,20,1,0,7,22,3,1055,638,1,0,0,9,25,5,1815,1275,18,0,0,8,8,4,1119,904,6,0,1,7,-21.0,1.0,6.0,NaN,166.0,-81.0,181.0,1,4,2,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,2449,0,4,1974,3,63


In [3]:
# Смотрим на количество признаков.

data.shape

(97230, 108)

In [4]:
# Определяем столбцы с финальными результатами и удаляем.

result_cols = ['duration', 
               'radiant_win', 
               'tower_status_radiant', 
               'tower_status_dire',
               'barracks_status_radiant',
               'barracks_status_dire']

X = data.drop(result_cols, axis=1)
X.shape

(97230, 102)

**Задание 2.**
Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [5]:
# Как видно, пропуски есть в нескольких признаках (их количество меньше размера таблицы - 97230).

X.count().sort_values().head(15)

first_blood_player2            53243
radiant_flying_courier_time    69751
dire_flying_courier_time       71132
first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
dire_bottle_time               81087
radiant_bottle_time            81539
radiant_first_ward_time        95394
dire_first_ward_time           95404
radiant_courier_time           96538
dire_courier_time              96554
d3_hero                        97230
d4_xp                          97230
d4_level                       97230
dtype: int64

In [6]:
# Ниже приведен список признаков, в которых есть пропущенные значения.

# Из описания признаков события "первая кровь" (first blood) известно, что если событие "первая кровь" 
# не успело произойти за первые 5 минут, то признаки принимают пропущенное значение.

# То есть, для четырёх первых признаков выполнилось условие, что в первые 5 минут не было события "первая кровь",
# отсюда и пропуски в данных.

X.columns[(X.count() < X.shape[0])].tolist()

['first_blood_time',
 'first_blood_team',
 'first_blood_player1',
 'first_blood_player2',
 'radiant_bottle_time',
 'radiant_courier_time',
 'radiant_flying_courier_time',
 'radiant_first_ward_time',
 'dire_bottle_time',
 'dire_courier_time',
 'dire_flying_courier_time',
 'dire_first_ward_time']

**Задание 3.** Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [7]:
# Заполняем пропуски нулями

X.fillna(0, inplace=True)

**Задание 4.** Какой столбец содержит целевую переменную? Запишите его название.

Наша задача - по первым 5 минутам игры предсказать, какая из команд победит. Соответственно, целевая переменная - `radiant_win` (1 - победила команда Radiant, 0 — иначе)

In [8]:
# выделим ответы

y = data['radiant_win']

**Задание 5.**
Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [9]:
# разбиения для кросс-валидации

cv = KFold(n_splits=5, shuffle=True)

In [10]:
scores = []
for n_estimators in [10, 20, 30, 50, 100]:
    start_time = datetime.datetime.now()
    score = cross_val_score(GradientBoostingClassifier(n_estimators=n_estimators), X, y, scoring='roc_auc', cv=cv)
    scores.append(score)
    print('Estimators: {}\tScore: {}\tTime elapsed: {}'.format(n_estimators, score.mean(),
                                                               datetime.datetime.now() - start_time))

Estimators: 10	Score: 0.6646135634467064	Time elapsed: 0:00:25.333265
Estimators: 20	Score: 0.6821016240528588	Time elapsed: 0:00:47.686765
Estimators: 30	Score: 0.6892768747201629	Time elapsed: 0:01:04.287864
Estimators: 50	Score: 0.6967461106056273	Time elapsed: 0:01:39.245864
Estimators: 100	Score: 0.706311290613572	Time elapsed: 0:03:02.808505


**Вопросы и ответы:**

1. Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?
    - Список признаков с пропусками: 'first_blood_time', 'first_blood_team', 'first_blood_player1', 'first_blood_player2', 'radiant_bottle_time', 'radiant_courier_time', 'radiant_flying_courier_time', 'radiant_first_ward_time', 'dire_bottle_time', 'dire_courier_time', 'dire_flying_courier_time', 'dire_first_ward_time'.
    - Из описания признаков события "первая кровь" (first blood) известно, что если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение. То есть, для четырёх первых признаков выполнилось условие, что в первые 5 минут не было события "первая кровь", отсюда и пропуски в данных.    
2. Как называется столбец, содержащий целевую переменную?
    - Наша задача - по первым 5 минутам игры предсказать, какая из команд победит. Соответственно, целевая переменная - radiant_win (1 - победила команда Radiant, 0 — иначе).
3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Какое качество при этом получилось?
    - Время: 0:01:04.287864
    - Качество: 0.6892768747201629
4. Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?
    - Есть смысл использовать больше 30 деревьев. При обучении обычно фиксируют число деревьев и подбирают 'learning_rate' и другие параметры. Для ускорения обучения можно ограничивать глубину деревьев 'max_depth'.

In [10]:
from itertools import product

scores = []
for learning_rate, n_estimators, max_depth in product([0.05, 0.1, 0.2], [10, 20, 30, 50], [2, 3]):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth)
    score = cross_val_score(clf, X, y, scoring='roc_auc', cv=cv)
    scores.append(score)
    print('Estimators: {}\tLR: {}\tDepth: {}\tScore: {}\tTime: {}'.format(n_estimators,
                                                                          learning_rate,
                                                                          max_depth,
                                                                          score.mean(),
                                                                          datetime.datetime.now() - start_time))

Estimators: 10	LR: 0.05	Depth: 2	Score: 0.6256037566334394	Time: 0:00:13.457995
Estimators: 10	LR: 0.05	Depth: 3	Score: 0.6390804623569887	Time: 0:00:21.583267
Estimators: 20	LR: 0.05	Depth: 2	Score: 0.6523564469503754	Time: 0:00:22.656091
Estimators: 20	LR: 0.05	Depth: 3	Score: 0.6669629414992674	Time: 0:00:38.873402
Estimators: 30	LR: 0.05	Depth: 2	Score: 0.6681267392259127	Time: 0:00:30.796723
Estimators: 30	LR: 0.05	Depth: 3	Score: 0.6776940550555197	Time: 0:00:59.117998
Estimators: 50	LR: 0.05	Depth: 2	Score: 0.6804187992502138	Time: 0:00:49.937964
Estimators: 50	LR: 0.05	Depth: 3	Score: 0.6875813859409398	Time: 0:01:28.099882
Estimators: 10	LR: 0.1	Depth: 2	Score: 0.6526392623457422	Time: 0:00:14.031997
Estimators: 10	LR: 0.1	Depth: 3	Score: 0.6654987237073042	Time: 0:00:27.165086
Estimators: 20	LR: 0.1	Depth: 2	Score: 0.6752897634710991	Time: 0:00:30.215344
Estimators: 20	LR: 0.1	Depth: 3	Score: 0.6824832576159974	Time: 0:00:48.290994
Estimators: 30	LR: 0.1	Depth: 2	Score: 0.682